### Test out the new logger function

First test was to make sure that the logger only initiates (or at the very least makes a message) only once. Had to make the logger metaclass=Singleton. Totally don't understand that. 

In [1]:
from fireatlas import FireConsts, settings, postprocess
from fireatlas.FireLog import logger
from dotenv import set_key
import os

2024-08-13 12:58:16,648 - fireatlas.FireLog - INFO - logger initialized!


Now let's test out it's ability to change the path of the file handler. Make its subdirectories defined by year and region. 

In [2]:
# in a typical script, there should be region and year already defined as global vars
# define them here
region = ['Zogg', 'Zogg_perimeter']
tst = [2020, 9, 15, 'AM']
location = None


Show it's file handler path intially, change it to a subdirectory, reset it.

In [3]:
print(logger.handlers)
newpath = postprocess.all_dir(tst, region, location)
logger.update_fh_dir(newpath)
print(logger.handlers)
logger.reset_fh()
print(logger.handlers)


[<StreamHandler stderr (INFO)>, <FileHandler D:\fireatlas\running.log (INFO)>]
[<StreamHandler stderr (INFO)>, <FileHandler D:\fireatlas\data\FEDSoutput-v3\Zogg\2020\running.log (INFO)>]
[<StreamHandler stderr (INFO)>, <FileHandler D:\fireatlas\running.log (INFO)>]


Next, need to show that the logger will change file handler path when the region and year changes. I suspect this will have no issues when done in serial, but question remains when done in parallel with `dask`. 

In [4]:
# create a list of "fires" to process
region_list = [['castle', 'castle_perim'], 
               ['caldor', 'caldor_perim'], 
               ['mosquito', 'mosquito_perim'], 
               ['zogg', 'zogg_perim']]
tst_list = [
    [2020, 8], [2021, 7], [2022, 8], [2020, 9]
]
location = None

In [5]:
# create a function to apply to each iteration
# does the logger properly log in the right subdirectory
# can it also log to the main directory when it's not region/year specific? 
def test_logger(logger, tst, region, location):
    # log into the main file handler
    logger.reset_fh()
    logger.info("Processing something for all fires.")

    # log into a subdirectory
    subdir = postprocess.all_dir(tst, region, location)
    logger.update_fh_dir(subdir)
    os.makedirs(subdir, exist_ok=True)
    logger.info(f"logging into the {region[0]} fire.")
    

Logger works great when used in serial. Check out the running.log files in the data directory.

In [6]:
for tst, region in zip(tst_list, region_list):
    test_logger(logger, tst, region, location)

2024-08-13 12:58:16,738 - fireatlas.FireLog - INFO - Processing something for all fires.
2024-08-13 12:58:16,742 - fireatlas.FireLog - INFO - logging into the castle fire.
2024-08-13 12:58:16,745 - fireatlas.FireLog - INFO - Processing something for all fires.
2024-08-13 12:58:16,747 - fireatlas.FireLog - INFO - logging into the caldor fire.
2024-08-13 12:58:16,748 - fireatlas.FireLog - INFO - Processing something for all fires.
2024-08-13 12:58:16,748 - fireatlas.FireLog - INFO - logging into the mosquito fire.
2024-08-13 12:58:16,756 - fireatlas.FireLog - INFO - Processing something for all fires.
2024-08-13 12:58:16,756 - fireatlas.FireLog - INFO - logging into the zogg fire.


Test out the logger when using `dask`!

In [7]:
from dask.distributed import Client
import dask.delayed
from functools import partial
import logging

# i need to set the level to error for the dask client. Have no clue how to not do this. 
logging.getLogger('client').setLevel(logging.ERROR)


# start dask client
client = Client(n_workers = 4, threads_per_worker = 1)

print(client.dashboard_link)

http://127.0.0.1:8787/status


In [8]:
# try it out 
futures = client.map(partial(test_logger, logger, location=location), tst_list, region_list)
results = client.gather(futures)

In [9]:
client.close()

In [13]:
# Print active loggers
sorted(logger.root.manager.loggerDict.keys())

logger.root.manager.loggerDict['client']




<CustomLogger fireatlas.FireLog (CRITICAL)>

In [12]:
import logging
logging.getLogger('client').setLevel(logging.CRITICAL)

In [ ]:

for logger_name in logging.root.manager.loggerDict:
    logger = logging.getLogger(logger_name)
    print(f"{logger_name}: {logger.level}")

In [5]:
# define env file
filenm = os.path.join(FireConsts.root_dir, '.env')
set_key(filenm, 'FEDS_LOG_SUBDIR', 'true')
set_key(filenm, 'FEDS_READ_LOCATION', 'local')
set_key(filenm, 'FEDS_INPUT_DIR', 'test')

(True, 'FEDS_INPUT_DIR', 'test')

In [6]:
print(settings.LOG_SUBDIR)
print(settings.READ_LOCATION)
print(settings.INPUT_DIR)

True
local
test


In [7]:
FireLog.all_dir_logger(tst, region, location)